# Solution of standard and fractional Fokker-Planck equation using spectral method
Authors: Aleksander Jakóbczyk, Jakub Koral, Hubert Woszczek

## Basic functions

In this section we define basic functions which will be used throughout this project.

In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from scipy.linalg import expm
from scipy.special import hermite, roots_hermite

from mittag_leffler import ml

sns.set_style("darkgrid")
sns.set_palette("rocket")

In [ ]:
def w_alpha(x, alpha): # weight function (see (9))
    return np.exp(alpha ** 2 * x ** 2)


def hermiteFunc(x, n, alpha): # Hermite function
    return 1 / math.sqrt(2 ** n * np.math.factorial(n)) * hermite(n)(alpha * x) * 1/w_alpha(x, alpha)


def f_delta_0(N, alpha, f_0, G_H_root=10): # coefficients of initial condition approximation (see (13-16) and (24))
    f_hat = np.zeros((N, 1))
    x_i, w_i = roots_hermite(G_H_root)
    for n in range(N):
        G_H_quad = np.sum(w_i * hermiteFunc(x_i, n, alpha) *
                          f_0(x_i) * w_alpha(x_i, alpha) * np.exp(x_i**2))
        f_hat[n] = alpha/np.sqrt(np.pi) * G_H_quad
    return f_hat


def M_f_ij(N, c, Y, alpha, u=1): # matrix connected with system of ODEs (see (28-30))
    M_f = np.zeros((N + 1, N + 1))
    for i in range(1, N + 2):
        n = i - 1
        for j in range(1, N + 2):
            if i == j:
                M_f[i - 1, j - 1] = n * Y
            elif i == j + 1:
                M_f[i - 1, j - 1] = alpha * u * np.sqrt(2 * n)
            elif i == j + 2:
                M_f[i - 1, j - 1] = (Y + 2 * alpha ** 2 *
                                     c) * np.sqrt(n * (n - 1))
    return M_f


def hermit_aprox(x, ts, f0_n, M_f, alpha, N): # approximates the solution given coefficients of initial condition (see (22) and (31))
    H = np.zeros((N+1, len(x)))
    for n in range(N+1):
        H[n] = hermiteFunc(x, n, alpha)
    f_t = np.zeros((len(ts), len(x)))
    for i, t in enumerate(ts):
        f_n = expm(M_f*t) @ f0_n
        f_t[i] = f_n.T @ H
    return f_t


def f(x, ts, N, alpha, f_0, M_f=None, G_H_root=10): # combines f_delta_0 and hermit_aprox functions
    if type(ts) == int:
        ts = [ts]
    f0_n = f_delta_0(N+1, alpha, f_0, G_H_root)
    f_t = hermit_aprox(x, ts, f0_n, M_f, alpha, N)
    if type(ts) == int:
        return ts[0]
    else:
        return f_t


def animation_function(i, x, ts, f_t, solution, ylim=None, xlim=None): # animation function
    t = ts[i]
    plt.clf()
    if xlim:
        plt.ylim(xlim)
    if ylim:
        plt.ylim(ylim)
    plt.plot(x, solution[i], label='exact solution', zorder=1)
    plt.scatter(x, f_t[i], marker='x', s=15, color='r',
                label='numerical solution', zorder=2)
    plt.title(
        fr'Exact and numerical solution of FPE for $x\in({min(x)},{max(x)}), t={np.round(t, 2):.2f}$')
    plt.legend()


## Introduction

The Fokker-Planck equation (often abbrevaited as FPE) is a partial differential equation that describes the time evolution of the probability density function of the velocity of a particle under the influence of drag and random forces. It is named after Adriaan Fokker and Max Planck, who derived it in 1914 and 1917 respectively. They used it to describe the Brownian motion of a free particle. Fokker-Planck equation has multiple applications in natural sciences, information theory, graph theory, data science, finance, economics, etc.

Various methods of solutions for the FPE have been proposed in scientific literature. Analytic solutions of the FPE can be found in some special cases, but in general they are difficult to obtain. Therefore, numerical methods have become important in approximating the solutions of the FPE.

First part of this project consists of a short theoretical introduction. Then we consider the class of FPEs corresponding to Itô SDEs described by the following multidimensional model
\begin{equation}
\begin{cases}
dX_t=\mu(X_t,t)\,dt+\sigma(X_t,t)W_t,\\
X_{t_0}=X_0,
\end{cases}
\end{equation}
where $W_t$ is a Wiener process (also known as Brownian motion). The evolution of the probability denisty function $f(x,t)$ associated with the stochastic process $X_t$ is governed by the following FPE
\begin{equation}
    \partial_t f(x,t)=-\partial_x\left(\mu(x,t)f(x,t)\right)+\frac{1}{2}\partial_{xx}\left(a(x,t)f(x,t)\right),\tag{2}
\end{equation}
where $\sigma(X_t,t)=\sqrt{a(X_t,t)}$. 

Nextly, we show how the solution changes when the fractional derivative with respect to time is introduced, leading to farctional Fokker-Planck equation (FFPE). Finally, we provide some numerical evidence of important properties of spectral method.


## Theory

### Hermite polynomials

Hermite polynomials are a sequence of orthogonal polynomials. They can be defined from several different starting points. Two classical forms of these polynomials are called "physicists's" and "probabilist's" Hermite polynomials. In this project we use the former one. Using derivatives we can define Hermite polynomials by formula
\begin{equation}
    H_n(x)=(-1)^n e^{x^2}\frac{d^n}{dx^n}e^{-x^2},\quad n\geq 0.\tag{3}
\end{equation}
This sequence of polynomials also satisfies the recurrence relation
\begin{equation}
    H_{n+1}(x)=2xH_n(x)-2nH_{n-1}(x),\quad n\geq 1,\tag{4}
\end{equation}
with initial polynomials given as
\begin{equation}
    \begin{aligned}
        H_0(x)&=1,\\
        H_1(x)&=2x.
    \end{aligned}\tag{5}
\end{equation}
Thus the first few members of this sequence are
\begin{equation}
    \begin{aligned}
        H_0(x)&=1,\\
        H_1(x)&=2x,\\
        H_2(x)&=4x^2-2,\\
        H_3(x)&=8x^3-12x,\\
        H_4(x)&=16x^4-48x^2+12.
    \end{aligned}\tag{6}
\end{equation}

The Hermite polynomials are orthogonal with respect to the weight function $w(x)=e^{-x^2}$
\begin{equation}
    \int_{\mathbb{R}} H_n(x)H_m(x)e^{-x^2}\,dx=\sqrt{\pi}2^nn!\delta_{nm},\tag{7}
\end{equation}
where $\delta_{nm}$ is the Kronecker delta.

The Hermite polynomials are generally not suitable for our FPE due to their asymptotic behavior at infinities
\begin{equation}
    H_n(x)\sim\frac{\Gamma(n+1)}{\Gamma\left(\frac{n}{2}+1\right)}e^{\frac{x^2}{2}}\cos\left(\sqrt{2n+1}x-\frac{n\pi}{2}\right)\sim n^{\frac{n}{2}}e^{\frac{x^2}{2}}\cos\left(\sqrt{2n+1}x-\frac{n\pi}{2}\right),\quad |x|\longrightarrow\infty.\tag{8}
\end{equation}

### Hermite functions

Hermite functions are defined as follows
\begin{equation}
    \tilde{H}_n(x)=\frac{1}{\sqrt{2^nn!}}H_n(\alpha x)w^{-1}_{\alpha}(x),\tag{9}
\end{equation}
where $w_{\alpha}(x)=e^{\alpha^2x^2}$ is a weight function.
In contrast to the Hermite polynomials, the Hermite functions have the following asymptotic behaviour
\begin{equation}
\tilde{H}_n(x)\sim n^{-\frac{1}{4}}\cos\left(\sqrt{2n+1}x-\frac{n\pi}{2}\right),\quad |x|\longrightarrow\infty.\tag{10}
\end{equation}
Below we present the plot of first five Hermite polynomials and functions.

In [ ]:
fig = plt.figure(figsize=(18, 6))

plt.subplot(1, 2, 1)
x_1 = np.linspace(-2, 2, 1000)
for n in range(5):
    plt.plot(x_1, hermite(n)(x_1), label=fr'$H_{n}(x)$')

plt.xlabel(r'$x$')
plt.legend(loc='upper left')
plt.title('The first five Hermite polynomials')

plt.subplot(1, 2, 2)
x_2 = np.linspace(-6, 6, 1000)
for n in range(5):
    plt.plot(x_2, hermiteFunc(x_2, n, 1), label=r'$\tilde{H}$' + fr'$_{n}(x)$')

plt.xlabel(r'$x$')
plt.legend()
plt.title('The first five Hermite functions')
plt.suptitle('Comparison of first five Hermite polynomials and functions')
plt.show()


We observe that the set of functions $\left\{\tilde{H}_n(x), n\geq 0\right\}$ defines a $L^2_{w_\alpha}(\mathbb{R})$-orthogonal system with scalar product given by formula
\begin{equation}
    \left(\tilde{H}_n(x), \tilde{H}_m(x)\right)_{w_\alpha}=\frac{\sqrt{\pi}}{\alpha}\delta_{nm}.\tag{11}
\end{equation}
Therefore for all $y\in L^2_{w_\alpha}(\mathbb{R})$, we can write
\begin{equation}
    y(x)=\sum_{n=0}^\infty\hat{y}_n\tilde{H}(x),\tag{12}
\end{equation}
with the coefficients
\begin{equation}
    \hat{y}_n=\frac{\alpha}{\sqrt{\pi}}\int_{\mathbb{R}} y(x) \tilde{H}_n(x) w_{\alpha}(x)\,dx,\quad n\geq 0.\tag{13}
\end{equation}
We will use this fact to approximate initial condition of FPE.

### Gauss-Hermite quadrature

Gauss-Hermite quadrature is a form of Gauss quadrature that uses Hermite polynomials. It is used to approximate integrals of the following kind
\begin{equation}
    \int_{\mathbb{R}}e^{-x^2}f(x)\,dx.\tag{14}
\end{equation}
In this case
\begin{equation}
    \int_{\mathbb{R}}e^{-x^2}f(x)\,dx\approx\sum_{i=1}^n w_if(x_i),\tag{15}
\end{equation}
where $n$ is the number of sample points used. The $x_i$, where $i=1,2,\ldots,n$ are the roots of the Hermite polynomial $H_n(x)$. The associated $w_i$ weights are given by
\begin{equation}
    w_i=\frac{2^{n-1}n!\sqrt{\pi}}{n^2\left[H_{n-1}(x_i)\right]^2}.\tag{16}
\end{equation}

### Caputo fractional derivative

Caputo derivatve is one of the possible forms of fractional derivative. It is defined as
\begin{equation}
    D^{\nu}_tf(t)=\frac{1}{\Gamma(n-\mu)}\int_0^t(t-\tau)^{n-\nu-1}f^{n}(\tau)\,d\tau,\quad n=\lceil\nu\rceil.\tag{17}
\end{equation}
Its Laplace transform is given by
\begin{equation}
    \mathcal{L}\left\{D^{\nu}_tf(t)\right\}(s)=s^{\nu}F(s)-\sum_{k=0}^{n-1}s^{\nu-k-1}f^{(k)}(0), \quad \mathcal{L}\left\{f(t)\right\}(s)=F(s).\tag{18}
\end{equation}

### Mittag-Leffler function

The Mittag-Leffler function $E_{\nu, \theta}$ is a special function, which depends on two complex parameters $\nu$ and $\theta$. It may be defined by the following series provided that, the real part of $\nu$ is strictly positive:
\begin{equation}
    E_{\nu, \theta}(z)=\sum_{k=0}^{\infty}\frac{z^k}{\Gamma(\nu k+\theta)}.\tag{19}
\end{equation}
When $\theta=1$, it is abbreviated as $E_{\nu}(z)=E_{\nu, 1}(z)$. In the case when both $\nu$ and $\theta$ are real and positive, the series converges for all values of the argument $z$, so the Mittag-Leffler function is an entire function. One can also easily prove that the following formula with Laplace transform $\mathcal{L}$
\begin{equation}
    \mathcal{L}\left\{t^{\theta-1}E_{\nu, \theta}(at^{\nu})\right\}(s)=\frac{s^{\nu-\theta}}{s^{\nu}-a}.\tag{20}
\end{equation}

## The Fokker-Planck equation for Itô processes

### Algorithm description

We consider a FPE corresponding to a representative stochastic process given by the Ornstein-Uhlenbeck process, and for simplicity we focus on a case in which the function $\mu(x,t)=\gamma x+u$ and $a(x,t)=2c$, where $\gamma<0$ and $u, c>0$ are constants. In this case the FPE problem is given by
\begin{equation}
    \begin{cases}
    \partial_t f(x,t)=-\partial_x((\gamma x+u)f(x,t))+c\partial_{xx}f(x,t),&\quad (x,t)\in\mathbb{R}\times (0,T);\\
    f(x,0)=\rho(x),&\quad x\in\mathbb{R}.\tag{21}
    \end{cases}
\end{equation}

The probability density $f$ is approximated in the space of Hermite functions as follows
\begin{equation}
    f(x,t)=\sum_{n=0}^\infty \hat{f}_n(t)\tilde{H}_n(x).\tag{22}
\end{equation}
The initial condition (probability density function of $X_{t_0}$) is also represnted  the Hermite functions space by
\begin{equation}
    \rho(x)=\sum_{n=0}^\infty \hat{f}^0_n\tilde{H}_n(x),\tag{23}
\end{equation}
where
\begin{equation}
    \hat{f}^0_n=\frac{\alpha}{\sqrt{\pi}}\int_{\mathbb{R}} y(x) \tilde{H}_n(x) w_{\alpha}(x)\,dx,\quad n\geq 0.\tag{24}
\end{equation}
Introducing the Hermite expansion for $f$ into the equation $(21)$, we have
\begin{equation}
    \frac{d}{dt}\hat{f}_n(t)=n\gamma\hat{f}_n(t)+\alpha u\sqrt{2u}\hat{f}_{n-1}(t)+(\gamma+2\alpha^2c)\sqrt{n(n-1)}\hat{f}_{n-2}(t),\tag{25}
\end{equation}
for $n\geq 0$ and with $\hat{f}_{-1}(t)=\hat{f}_{-2}(t)\equiv 0$. This equation represents an infinite system of ODEs. We truncate it by considering the approximation
\begin{equation}
    \left[\hat{f}_{0}(t), \hat{f}_{1}(t), \ldots \right]\approx\left[\hat{f}_{\Delta,0}(t), \hat{f}_{\Delta,1}(t), \ldots, \hat{f}_{\Delta,N}(t)\right]\tag{26}
\end{equation}
Therefore, the system of ODEs which we solve is as follows
\begin{equation}
    \begin{cases}
        \frac{d}{dt}\hat{f}_{\Delta, n}(t)=n\gamma\hat{f}_{\Delta, n}(t)+\alpha u\sqrt{2u}\hat{f}_{\Delta, n-1}(t)+(\gamma+2\alpha^2c)\sqrt{n(n-1)}\hat{f}_{\Delta, n-2}(t),\\
        \hat{f}_{\Delta, n}(0)=\hat{f}^0_n,\tag{27}
    \end{cases}
\end{equation}
for $0\leq n\leq N, 0\leq t\leq T$ and $\hat{f}_{\Delta, -1}(t)=\hat{f}_{\Delta, -2}(t)\equiv 0$. This corresponds to a Galerkin projection of $f(\cdot, t)$ onto the Hermite approximation space: $\operatorname{span}\left\{\tilde{H}_n(x), n\geq 0\right\}$. Writing this system of ODEs in the matrix form we obtain
\begin{equation}
    \frac{d\hat{f}_{\Delta}}{dt}=M_f\hat{f}_{\Delta},\tag{28}
\end{equation}
where
\begin{equation}
    \hat{f}_\Delta=\left[\hat{f}_{\Delta,0}(t), \hat{f}_{\Delta,1}(t), \ldots, \hat{f}_{\Delta,N}(t)\right]^T\tag{29}
\end{equation}
and $M_f$ is a $(N+1)\times(N+1)$ three-diagonal matrix defined by the following formula
\begin{equation}
    \left(M_f\right)_{ij}=
    \begin{cases}
        n\gamma, &i=j,\\
        \alpha u\sqrt{2n}, &i=j+1,\\
        (\gamma+2\alpha^2c)\sqrt{n(n-1)}, &i=j+2,\\
        0, &\text{otherwise},
    \end{cases}\tag{30}
\end{equation}
where $n=i-1$. Notice that, the first row in $M_f$ consists of zeros. Since the matrix $M_f$ is triangular, it has $N+1$ distinct eigenvalues. Therefore $M_f$ is diagonalizable and the system of ODEs has the solution
\begin{equation}
    \hat{f}_{\Delta}=e^{M_f t}\hat{f}^0_n.\tag{31}
\end{equation}
Lastly, we specify the value of paramter $\alpha$. To approximate qaussian type functions $e^{-sx^2}$, the scaling factor $\alpha$ must satisfy $\alpha\in(0,\sqrt{2s})$. The stationary solution ($\partial_t f=0$) of $(21)$ is as follows
\begin{equation}
    f(x)=C_0\exp\left(\frac{\gamma^2}{2c}x^2+\frac{u}{c}x\right),\tag{32}
\end{equation}
where $C_0$ is a constant. Comparing the stationary solution with the weight function $w_{\alpha}(x)$, while $u=0$, motivates us to set $\alpha=\sqrt{-\frac{\gamma}{2c}}$.

### Examples

In this section we consider few interesting examples of solutions of FPE with different initial conditions.

#### Solution for $f(x,0)=\delta(x)$

The general solution of
\begin{equation}
    \begin{cases}
    \partial_t f(x,t)=-\partial_x((\gamma x+u)f(x,t))+c\partial_{xx}f(x,t),&\quad (x,t)\in\mathbb{R}\times (0,T);\\
    f(x,0)=\delta(x),&\quad x\in\mathbb{R},\tag{33}
    \end{cases}
\end{equation}
is a gaussian distribution with mean $\tilde{\mu}(t)=\frac{u}{\gamma}\left(e^{\gamma t}-1\right)$ and variance $\tilde{\sigma}^2(t)=\frac{c}{\gamma}\left(e^{2\gamma t}-1\right)$, that is
\begin{equation}
    f(x, t)=\frac{1}{\sqrt{2\pi\tilde{\sigma}^2(t)}}\exp\left(-\frac{(x-\tilde{\mu}(t))^2}{2\tilde{\sigma}^2(t)}\right)\tag{34}
\end{equation}
Since it is impossible to represent the Dirac delta function $\delta(0)$ by Hermite functions we have to apply a shift in time to obtain a gaussian function as initial condition. We consider time $t=1$ as a starting time of the process. Initial condition coefficients can be computed analytically, using the properties of Dirac delta we obtain
\begin{equation}
    f_n^0=\frac{\alpha}{\sqrt{n}}\int_\mathbb{R}\delta(x)\tilde{H}_n(x)w_{\alpha}(x)\,dx=\frac{\alpha}{\sqrt{n}}\tilde{H}_n(0),\quad n\geq0.\tag{35}
\end{equation}
We set parameters $c=1$, $\gamma=-1$, $\alpha=\sqrt{-\frac{\gamma}{2c}}$ in accordance with $(33)$ and $N=15$. Results are presented as animation below.


In [ ]:
def f_exact(x, t, c, gamma, u):
    mu = u / gamma * (np.exp(gamma * t) - 1)
    sigma2 = c / gamma * (np.exp(2 * gamma * t) - 1)
    return 1 / np.sqrt(2 * np.pi * sigma2) * np.exp(-(x - mu) ** 2 / (2 * sigma2))


In [ ]:
c = 1
gamma = -1
alpha = np.sqrt(-gamma / (2 * c))

u = 2
N = 15
x = np.linspace(-5, 5, 100)
ts = np.linspace(1, 5, 101)


f0_n = alpha/np.sqrt(np.pi) * \
    np.array([[hermiteFunc(0, n, alpha)] for n in range(N+1)])
M_f = M_f_ij(N, c, gamma, alpha, u)
f_t = hermit_aprox(x, ts, f0_n, M_f, alpha, N)
solution = np.array([f_exact(x, t, c, gamma, u) for t in ts])

figure = plt.figure(figsize=(9, 6))
animation = FuncAnimation(figure, func=animation_function, fargs=[
                        x, ts, f_t, solution, (-0.1, 0.5)], frames=len(ts), interval=100)
html = HTML(animation.to_jshtml())
display(html)
plt.close()


Difference between exact and numerical solutions are indistinguishable for $x\in(-5,5)$ and $t\in[1,5]$.

#### Solution for $f(x,0)=\exp\left(-\frac{x^2}{2}\right)\left(1+\cos\left(\frac{\pi}{2}x\right)\exp\left({\frac{\pi^2}{8}}\right)\right)$

This example comes from doctoral thesis $[1]$. We notice that its solution does not define a probability density. Nevertheless it can also solved using this spectral method.

Obtaining an analytical formula for coefficients $\hat{f}_0^n$ would be challenging to say the least, so we shall use Hermite quadrature with $100$ sample points to approximate it. We maintain the same parameters as before, except for $N$, which is set to $10$. Again we present results as animation.

In [ ]:
def f_exact(x, t):
    return np.exp(-x**2/2) * (1 + np.cos(np.pi/2 * x * np.exp(-t)) * np.exp(np.pi**2 / 8 * np.exp(-2 * t)))

In [ ]:
c = 1
Y = -1
alpha = np.sqrt(-Y / (2 * c))
u = 0
N = 10

figure = plt.figure(figsize=(9, 6))
ts = np.linspace(0, 1, 101)
x = np.linspace(-5, 5, 100)
M_f = M_f_ij(N, c, Y, alpha, u)
f_t = f(x, ts, N, alpha, lambda x: f_exact(x, 0), M_f, G_H_root=100)
solution = np.array([f_exact(x, t) for t in ts])

animation = FuncAnimation(figure, func=animation_function, fargs=[
                        x, ts, f_t, solution, (-1, 5)], frames=len(ts), interval=100)
html = HTML(animation.to_jshtml())
display(html)
plt.close()


Once again we achieve a very good approximation of the solution of FPE.

### Comparison with numerical solution of SDE

Additionaly, example with $f(0,x)=\delta(x)$ is solved using Runge-Kutta and Milstein methods. Histograms of trajectories are compared with results obtained using sepctral method applied to the corresponding FPE.

#### Runge-Kutta method for SDE $[3]$ <t style="font-variant: small-caps">

Consider the stochastic differential equation
\begin{equation}
\begin{cases}
dX_t=\mu(X_t)\,dt+\sigma(X_t)\,dW_t,\\
X_{t_0}=X_0,
\end{cases}\tag{36}
\end{equation}
with initial condition $X_{0}=x_{0}$, where 
$W_{t}$ denotes standard Brownian motion. 

Suppose that we wish to solve $(36)$ on some finite time horizon $[0, T]$. Then the Runge-Kutta approximation is defined as follows. First, we partition the $[0, T]$ time interval into $N$ subintervals of length $\Delta t$. Namely, $0=\tau_0<\tau_1< \ldots <\tau_n = T$, where $\tau_n = n \Delta t$, $\Delta t = \frac{T}{n}$. Next we set $Y_0 = x_0$. Finally, we define $Y_{n+1}$ as $Y_{n+1} = Y_n + \mu(Y_n)\Delta t + \sigma(Y_n)\Delta W_n + \frac{1}{2}(\sigma(\Upsilon_n) - \sigma(Y_n)((\Delta W_n)^2 - \Delta t)\Delta t^{-\frac{1}{2}}$, where $\Delta W_n = W_{\tau_{n+1}} - W_{\tau_n}$ and $\Upsilon_n =  Y_n + \mu(Y_n)\Delta t + \sigma(Y_n)\sqrt{\Delta t}$. Then $Y_n$ is the numerical solution to given SDE.

In [ ]:
def f_exact(x, t, c, gamma, u):
    mu = u / gamma * (np.exp(gamma * t) - 1)
    sigma2 = c / gamma * (np.exp(2 * gamma * t) - 1)
    return 1 / np.sqrt(2 * np.pi * sigma2) * np.exp(-(x - mu) ** 2 / (2 * sigma2))

In [ ]:
def runge_kutta(T, N, Y_0, mu, sigma, n):
    dt = T/N
    trajectories = np.zeros((n, N))
    for j in range(n):
        ys = np.zeros(N)
        ys[0] = Y_0
        trajectories[j][0] = Y_0
        for i in range(1, N):
            y = ys[i - 1]
            dw_ = np.random.normal(loc=0, scale=np.sqrt(dt))
            Y = y + mu(y)*dt + sigma(y) * np.sqrt(dt)
            ys[i] = y + mu(y)*dt + sigma(y) * dw_ + 0.5 * \
                (sigma(Y) - sigma(y)) * (dw_**2 - dt)*np.sqrt(dt)
            trajectories[j][i] = y + mu(y)*dt + sigma(y) * dw_ + \
                0.5 * (sigma(Y) - sigma(y)) * (dw_**2 - dt)*np.sqrt(dt)
    return trajectories


In [ ]:
def milstein(T, N, Y_0, mu, sigma, sigma_prime, n):
    dt = T/N
    trajectories = np.zeros((n, N))
    for j in range(n):
        ys = np.zeros(N)
        ys[0] = Y_0
        trajectories[j][0] = Y_0
        for i in range(1, N):
            y = ys[i - 1]
            dw_ = np.random.normal(loc=0, scale=np.sqrt(dt))
            ys[i] = y + mu(y)*dt + sigma(y) * dw_ + 0.5 * \
                sigma(y) * sigma_prime(y) * (dw_**2 - dt)
            trajectories[j][i] = y + mu(y)*dt + sigma(y) * dw_ + \
                0.5 * sigma(y) * sigma_prime(y) * (dw_**2 - dt)
    return trajectories


In [ ]:
def animation_function_3(i, x, ts, f_t, solution, SDE, ylim=None, xlim=None, method=None):
    t = ts[i]
    plt.clf()
    if xlim:
        plt.xlim(xlim)
    if ylim:
        plt.ylim(ylim)
    plt.plot(x, solution[i], label='exact solution', zorder=1)
    plt.scatter(x, f_t[i], marker='x', s=15, color='r',
                label='numerical solution', zorder=2)
    plt.hist(SDE[:, i+25], density=True,
            bins=20, label='SDE solution')
    
    plt.title(
        fr'Exact, numerical and SDE (by {method} method) solutions of FPE for $x\in(-5,5), t={t:.2f}$')
    plt.legend()

In [ ]:
c = 1
gamma = -1
alpha = np.sqrt(-gamma / (2 * c))
u = 2
N = 15
x = np.linspace(-5, 5, 100)
ts = np.linspace(1, 5, 101)
X_0 = 0

f0_n = alpha/np.sqrt(np.pi) * w_alpha(X_0, alpha) * \
    np.array([[hermiteFunc(X_0, n, alpha)] for n in range(N+1)])
M_f = M_f_ij(N, c, gamma, alpha, u)
f_t = hermit_aprox(x, ts, f0_n, M_f, alpha, N)
solution = np.array([f_exact(x, t, c, gamma, u) for t in ts])


def mu_test(x): return gamma*x + u
def sigma_test(x): return np.sqrt(2*c)
def sigma_prime_test(x): return 0

test_runge_kutta = runge_kutta(5, 126, X_0, mu_test, sigma_test, 10000)


In [ ]:
figure = plt.figure(figsize=(9, 6))
animation = FuncAnimation(figure, func=animation_function_3, fargs=[
                        x, ts, f_t, solution, test_runge_kutta, (-0.1, 0.5), (-6,6) , "Runge Kutta"], frames=len(ts), interval=100)
html = HTML(animation.to_jshtml())
display(html)
plt.close()

Histograms of trajectories obtained using Runge-Kutta method and results obtained using sepctral method fit almost perfectly.

#### Milstein method $[4]$ <t style="font-variant: small-caps">

Another method to solve $(36)$ is called Milstein method. Again partition the time interval $[0, T]$ into $N$ equal subintervals of width $\Delta t$ and set $Y_0=x_0$. Recursively put $Y_{n+1}=Y_n+\mu(Y_n)\Delta t+\sigma(Y_n)\Delta W_n+\frac{1}{2}\sigma(Y_n)\sigma'(Y_n)\left[(\Delta W_n)^2-\Delta t\right]$, where $\Delta W_n=W_{\tau_{n+1}}-W_{\tau_{n}}$ are independent and identically distributed normal random variables with expected value zero and variance $\Delta t$.

In [ ]:
test_milstein = milstein(5, 126, X_0, mu_test, sigma_test, sigma_prime_test, 10000)
figure = plt.figure(figsize=(9, 6))
animation = FuncAnimation(figure, func=animation_function_3, fargs=[
                        x, ts, f_t, solution, test_milstein, (-0.1, 0.5), (-6,6) , "Milstein"], frames=len(ts), interval=100)
html = HTML(animation.to_jshtml())
display(html)
plt.close()

Histograms of trajectories obtained using Milstein method and results obtained using sepctral method fit almost perfectly.

## Fractional Fokker-Planck equation

We now turn to fractional Fokker-Planck equation (abbrevaited as FFPE). It is used to model subdiffusion, the movement of particles in porous medium and gels. FFPE is defined by:
\begin{equation}
    \begin{cases}
    D_t^{\nu} f(x,t)=-\partial_x((\gamma x+u)f(x,t))+c\partial_{xx}f(x,t),&\quad (x,t)\in\mathbb{R}\times (0,T);\\
    f(x,0)=\rho(x),&\quad x\in\mathbb{R},
    \end{cases}\tag{37}
\end{equation}
where $D_t^{\nu} f(x,t)$ is a Caputo fractional derivative of $f(x,t)$ with respect to $t$ and order $\nu\in(0,1)$. Following the same logic as before we obtain a system of FDEs
\begin{equation}
    \begin{cases}
        D_t^{\nu}\hat{f}_\Delta=M_f\hat{f}_\Delta,\\
        \hat{f}_\Delta(0)=\hat{f}_n^0.
    \end{cases}\tag{38}
\end{equation}
We solve this problem using Laplace transform and equations $(18)$ and $(20)$
\begin{equation}
    \begin{aligned}
        &\mathcal{L}\left\{D_t^{\nu}\hat{f}_\Delta\right\}(s)-M_f\mathcal{L}\left\{\hat{f}_\Delta\right\}(s)=0\\
        &s^{\nu}\hat{F}_\Delta-\sum_{k=0}^{n-1}s^{\nu-k-1}\hat{f}_\Delta^{(k)}(0)-M_f\hat{F}_\Delta=0,\quad \hat{F}_\Delta(s)=\mathcal{L}\left\{\hat{f}_\Delta(t)\right\}(s),\\
        &n=1\Longrightarrow s^{\nu}\hat{F}_\Delta s^{\nu-1}\hat{f}_\Delta(0)-M_f\hat{F}_\Delta=0\\
        &\hat{F}_\Delta=\frac{s^{\nu-1}}{s^{\nu}-M_f}\hat{f}_n^0,\\
        &\hat{f}_\Delta=\mathcal{L}^{-1}\left\{\frac{s^{\nu-1}}{s^{\nu}-M_f}\right\}\hat{f}_n^0,\\
        &\hat{f}_\Delta=E_{\nu}\left(M_f t^{\nu}\right)\hat{f}_n^0,
    \end{aligned}\tag{39}
\end{equation}
where $E_{\nu}$ is the Mittag-Leffler function. We are not aware of any existing implementation of this special function in Python for matrices. However we found one for vectors. As we mentioned before our martix $M_f$ is diagonalizable. We denote the vector of its eigenvalues as $\vec{\lambda}=(\lambda_1, \lambda_2, \ldots, \lambda_{N+1})$. Thus we can simply apply the vector Mittag-Leffler function on $M_f$ by formula
\begin{equation}
    E_\nu(M_f)=P^{-1}\operatorname{diag}\left(E_\nu(\vec{\lambda})\right)P,
\end{equation}\tag{40}
where by $\operatorname{diag}\left(E_\nu(\vec{\lambda})\right)$ we understand a diagonal matrix filled with elements of vector $E_\nu(\vec{\lambda})$.

Below we present how the solution of FFPE differs from the solution of standard FPE for various values of $\nu$.

In [ ]:
def f_exact(x, t, c, gamma, u):
    mu = u / gamma * (np.exp(gamma * t) - 1)
    sigma2 = c / gamma * (np.exp(2 * gamma * t) - 1)
    return 1 / np.sqrt(2 * np.pi * sigma2) * np.exp(-(x - mu) ** 2 / (2 * sigma2))


In [ ]:
def f0_n(N, alpha):
    result = np.zeros((N + 1, 1))
    for n in range(N + 1):
        result[n] = alpha / np.sqrt(np.pi) * hermiteFunc(0, n, alpha)
    return result


In [ ]:
def ML(X, alpha):
    eigval, P = np.linalg.eig(X)
    D = np.diag(ml(eigval, alpha))

    return np.matmul(np.matmul(P, D), np.linalg.inv(P))


In [ ]:
def ffrac(x, t, N, c, gamma, alpha, u, nu):
    H = np.zeros((N + 1, len(x)))

    for n in range(N + 1):
        H[n] = hermiteFunc(x, n, alpha)

    return np.matmul(np.matmul(ML(M_f_ij(N, c, gamma, alpha, u) * t ** nu, nu), f0_n(N, alpha)).T, H).T


In [ ]:
c = 1
gamma = -1
alpha = np.sqrt(-gamma / (2 * c))
u = 2
N = 15

x = np.linspace(-5, 5, 100)
t = np.arange(1, 5.1, 0.1)
nu = [0.3, 0.5, 0.7, 0.9]

figure = plt.figure(figsize=(18, 12))


def animation_function_2(i):
    plt.clf()

    plt.subplot(2, 2, 1)
    plt.ylim(-0.1, 0.5)
    plt.plot(x, f_exact(x, t[i], c, gamma, u),
            label=r'exact solution of FPE ($\nu=1$)', zorder=1)
    plt.scatter(x, ffrac(x, t[i], N, c, gamma, alpha, u, nu[0]),
                marker='x', s=15, color='r', label='numerical solution', zorder=2)
    plt.title(
        fr'Exact and numerical solution of FFPE for $x\in(-5,5), t={np.round(t[i], 1)}, \nu={nu[0]}$')
    plt.legend(loc='upper left')

    plt.subplot(2, 2, 2)
    plt.ylim(-0.1, 0.5)
    plt.plot(x, f_exact(x, t[i], c, gamma, u),
            label=r'exact solution of FPE ($\nu=1$)', zorder=1)
    plt.scatter(x, ffrac(x, t[i], N, c, gamma, alpha, u, nu[1]),
                marker='x', s=15, color='r', label='numerical solution', zorder=2)
    plt.title(
        fr'Exact and numerical solution of FFPE for $x\in(-5,5), t={np.round(t[i], 1)}, \nu={nu[1]}$')
    plt.legend(loc='upper left')

    plt.subplot(2, 2, 3)
    plt.ylim(-0.1, 0.5)
    plt.plot(x, f_exact(x, t[i], c, gamma, u),
            label=r'exact solution of FPE ($\nu=1$)', zorder=1)
    plt.scatter(x, ffrac(x, t[i], N, c, gamma, alpha, u, nu[2]),
                marker='x', s=15, color='r', label='numerical solution', zorder=2)
    plt.title(
        fr'Exact and numerical solution of FFPE for $x\in(-5,5), t={np.round(t[i], 1)}, \nu={nu[2]}$')
    plt.legend(loc='upper left')

    plt.subplot(2, 2, 4)
    plt.ylim(-0.1, 0.5)
    plt.plot(x, f_exact(x, t[i], c, gamma, u),
            label=r'exact solution of FPE ($\nu=1$)', zorder=1)
    plt.scatter(x, ffrac(x, t[i], N, c, gamma, alpha, u, nu[3]),
                marker='x', s=15, color='r', label='numerical solution', zorder=2)
    plt.title(
        fr'Exact and numerical solution of FFPE for $x\in(-5,5), t={np.round(t[i], 1)}, \nu={nu[3]}$')
    plt.legend(loc='upper left')


animation = FuncAnimation(
    figure, func=animation_function_2, frames=len(t), interval=100)
html = HTML(animation.to_jshtml())
display(html)
plt.close()

We observe that the closer the order of derivative $\nu$ is to $1$, the more similar solutions are.

## Properties of spectral method

### Conservativity

Important property of the numerical scheme is that the Hermite spectral discretization provides conservativity, it means that
\begin{equation}
    \int_\mathbb{R} f_\Delta (x,t)\,dx = \int_\mathbb{R} f_\Delta (x,0)\,dx.\tag{41}
\end{equation}
We confirm this property numerically.

In [ ]:
def f_exact(x, t):
    return np.exp(-x**2/2) * (1 + np.cos(np.pi/2 * x * np.exp(-t)) * np.exp(np.pi**2 / 8 * np.exp(-2 * t)))

In [ ]:
c = 1
Y = -1
alpha = np.sqrt(-Y / (2 * c))
u = 0
x = np.linspace(-5, 5, 100)

ts = np.linspace(0, 2, 100)
dx = 0.1
xs = np.arange(-5, 5, dx)
df_conservativity = pd.DataFrame()
for N in np.arange(0, 15, 2, dtype=int):
    M_f = M_f_ij(N, c, Y, alpha, u)
    f_t = f(x, ts, N, alpha, lambda x: f_exact(x, 0), M_f, G_H_root=100)
    _df = pd.DataFrame(np.sum(f_t, 1)*dx, columns=["aprox"])
    _df["t"] = ts
    _df["N"] = N
    df_conservativity = pd.concat([df_conservativity, _df])


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18, 6))
sns.lineplot(df_conservativity, x="t", y="aprox", hue="N", ax=ax[0])
sns.lineplot(df_conservativity, x="t", y="aprox", hue="N", ax=ax[1])
ax[0].set_ylim(0, 6)
ax[0].set_ylabel("$\\int_\\mathbb{R} \\hat f_\\Delta (x,t) dx$")
ax[0].set_title("Conservativity approximation")
ax[1].set_ylabel("")
ax[1].set_title("Zoom")

plt.show()

We observe that for all values of $N$ the property of conservativity holds.

### Spectral convergence

Substituting the Hermite expansion into the FPE results in an infinite system of linear ODEs. Corresponding to this system, there is a matrix $M_\infty$, which is lower triangular. To have a practical scheme, we have to truncate this matrix, or equivalently, consider some truncated system of ODEs. However, this truncation is a source of error in our discretization scheme. It can be show that the approximation for the state variable is spectrally convergent:

***Lemma:*** *If $f \in L^\infty(0,T,H_{w_\alpha}^t(\mathbb{R})), r > 1$, then for all $t \in [0,T]$ the following holds*
\begin{equation}
        ||f(\cdot ,t) - f_\Delta (\cdot ,t)||_{w_\alpha}^2 = O(N^{-r})\tag{42}.
\end{equation}

In [ ]:
def approx_norm_w_alpha(f_t, x, alpha):
    dx = x[1] - x[0]
    return math.sqrt(np.sum((f_t)**2*w_alpha(x, alpha)) * dx)


In [ ]:
c = 1
Y = -1
u = 0

ts = np.linspace(0, 2, 10)
dx = 0.1
xs = np.arange(-5, 5, dx)
Ns = range(0, 39)
df_convergent = pd.DataFrame(columns=[*Ns, "alpha", "t"])


def f_0(x): return f_exact(x, 0)


for alpha in [0.3, 0.4, 0.6, 0.65, np.sqrt(-Y / (2 * c)), 0.75, 0.8, 0.9, 1]:
    for t in ts:
        norms = np.zeros(len(Ns))
        for i, N in enumerate(Ns):
            M_f = M_f_ij(N, c, Y, alpha, u)
            f_t = f(x, [t], N, alpha, f_0, M_f, G_H_root=100)
            solution = f_exact(x, t)
            norms[i] = approx_norm_w_alpha(f_t[0] - solution, x, alpha)
        norm_df = pd.DataFrame([norms**2])
        norm_df["alpha"] = alpha
        norm_df["t"] = t
        df_convergent = pd.concat([df_convergent, norm_df], axis=0)


In [ ]:
melt_df = df_convergent.melt(["t", "alpha"])

melt_df["alpha"] = melt_df["alpha"].round(4)
melt_df["t"] = melt_df["t"].round(3)
melt_df.rename(columns={"variable": "N"}, inplace=True)
g = sns.FacetGrid(melt_df, col="alpha", col_wrap=3, hue="t",  height=4)
g.map(sns.lineplot, "N", "value")
g.add_legend()
for ax in g.axes:
    ax.set_ylabel("$||f - f_\\Delta ||^2_{w_\\alpha}$")
    ax.set_title(ax.get_title().replace("alpha", "$\\alpha$"))
plt.yscale("log")
g.fig.subplots_adjust(top=0.93)
g.fig.suptitle("Spectral convergence")
plt.show()

Considering the above plots, we can conclude that the $\alpha$ parameter has a considerable impact on the type of convergence. 
By making a good choice of the $\alpha$ parameter, we can achieve convergence much better than algebraic.
Even supergeometric convergence can be obtained in the best case, when $\alpha$ is chosen in accordance with considerations following from $(32)$. 


### Time convergence

Furthermore, we can analyze the convergence of the algorithm over time.

In [ ]:
N_lim = 30
g = sns.FacetGrid(melt_df[melt_df["N"] < N_lim],
                col="alpha", col_wrap=3, hue="N",  height=4)
g.map(sns.lineplot, "t", "value")
g.add_legend()
for ax in g.axes:
    ax.set_ylabel("$||f - f_\\Delta ||^2_{w_\\alpha}$")
    ax.set_title(ax.get_title().replace("alpha", "$\\alpha$"))
plt.yscale("log")
g.fig.subplots_adjust(top=0.93)
g.fig.suptitle("Time convergence")
plt.show()

As in the previous case, the convergence is strongly dependent on the choice of the $\alpha$ parameter. In the tested example, we see that geometric convergence can be achieved in the best case. 

### Stability

The following theorem provides an appropriate means to show that the Hermite discretization method is stable.

***Theorem:*** *Let $\hat{f}_\Delta(t)$ be the solution to*
\begin{equation}
    \frac{d}{dt}\hat{f}_\Delta(t) = M_f \hat{f}_\Delta(0).\tag{43}
\end{equation}
*Then there exists a constant $C_N>0$ such that for all $t>0$*
\begin{equation}
    ||\hat{f}^t_\Delta||_{w_{\alpha}} \le C_N ||\hat{f}_\Delta^0||_{w_{\alpha}}.\tag{44}
\end{equation}


In [ ]:
Y = -1
u = 0
c = 1

ts = np.linspace(0, 5, 50)
dx = 0.1
xs = np.arange(-5, 5, dx)
Ns = np.arange(2, 27, 8, dtype=int)


def f_0(x): return f_exact(x, 0)


df_stable = pd.DataFrame(columns=[*Ns, "alpha", "t"])
for alpha in [0.3, 0.4, 0.6, 0.65, np.sqrt(-Y / (2 * c)), 0.75, 0.8, 0.9, 1]:
    for t in ts:
        norms = np.zeros(len(Ns))
        for i, N in enumerate(Ns):
            M_f = M_f_ij(N, c, Y, alpha, u)
            norm_f_0 = approx_norm_w_alpha(f_0(x), x, alpha)
            f_t = f(x, [t], N, alpha, f_0, M_f, G_H_root=100)
            norm_f_t = approx_norm_w_alpha(f_t, x, alpha)
            norms[i] = norm_f_t / norm_f_0

        norm_df = pd.DataFrame([norms], columns=Ns)
        norm_df["alpha"] = alpha
        norm_df["t"] = t
        df_stable = pd.concat([df_stable, norm_df], axis=0)


In [ ]:
melt_df = df_stable.melt(["t", "alpha"])
melt_df["alpha"] = melt_df["alpha"].round(4)
melt_df["t"] = melt_df["t"].round(3)
melt_df.rename(columns={"variable": "N"}, inplace=True)

g = sns.FacetGrid(melt_df, col="N", col_wrap=2,  height=5, hue="alpha")
g.map(sns.lineplot, "t", "value")
g.add_legend()
for ax in g.axes:
    ax.set_ylabel(
        "$\\frac{||\\hat{f}_\\Delta^t||_{w_\\alpha}}{||f_\Delta^0||_{w_\\alpha}}$")
    ax.set_title(ax.get_title().replace("alpha", "$\\alpha$"))
g.fig.subplots_adjust(top=0.93)
g.fig.suptitle("Stablity")
plt.show()

In the present case, it is evident that all the solutions are upper bound. Consequently, we can observe that the simulation confirms the validity of the stability assumption.

### Additional numerical experiments

Finally we conduct some further numerical experments, presenting how the solution behaves for different sets of parametrs $N$, $t$ and $\alpha$.

In [ ]:
c = 1
Y = -1
u = 0
ts = np.linspace(0, 1, 10)
ts = [0, 0.5, 1]
xs = np.linspace(-5, 5, 100)
Ns = [2, 5, 10]
df_experiment = pd.DataFrame(columns=[*xs, "N", "alpha", "t"])

def f_0(x): return f_exact(x, 0)
for alpha in [0.5, np.sqrt(-Y / (2 * c)), 0.8]:
    for N in Ns:
        M_f = M_f_ij(N, c, Y, alpha, u)
        f_t = f(xs, ts, N, alpha, f_0, M_f, G_H_root=100)
        _df = pd.DataFrame(f_t, columns=xs)
        _df["alpha"] = alpha
        _df["N"] = N
        _df["t"] = ts
        df_experiment = pd.concat([df_experiment, _df])

In [ ]:
melt_df_experiment = df_experiment.melt(
    ["N", "alpha", "t"], var_name="x", value_name="f")
melt_df_experiment['alpha'] = np.round(melt_df_experiment["alpha"], 2)
style = {"marker": ["s", "^", "o"], "alpha": [0.9] * 3, "ls": ["", "", ""]}
g = sns.FacetGrid(melt_df_experiment, row="t", col="N", hue="alpha",
                margin_titles=True, height=3.5, hue_kws=style)
g.map(sns.lineplot, "x", "f")
for i, ax in enumerate(g.axes[0]):
    _df = pd.DataFrame([xs, f_exact(xs, ts[i//3])]).T
    _df.columns = ["xs", "f"]
    sns.lineplot(_df, x="xs", y="f", ax=ax)

g.fig.subplots_adjust(top=0.92)
g.fig.suptitle(r"Numerical (marks) and exact solution (solid line) to the FP equation with different $\alpha$ factors")
plt.show()

The graph above illustrates how the numerical solution converges to an exact solution as $t$ and $N$ increase.

In [ ]:
df = []
for t in [0, 0.5, 1]:
    for alpha in [0.5, np.sqrt(-Y / (2 * c)), 0.8]:
        for N in Ns:
            _df_experiment = df_experiment[(df_experiment["t"] == t) &
                                           (df_experiment['alpha'] == alpha) &
                                           (df_experiment['N'] == N)
                                           ]
            norm = approx_norm_w_alpha(np.array(_df_experiment[xs] - f_exact(xs, t)), xs, 0)
            df.append([t, alpha, N, norm])
df = pd.DataFrame(df, columns=["t", "alpha", "N", "norm"])
pivot_df = df.pivot_table(df, index=["t", "alpha"], columns="N")
pivot_df.rename(columns={"norm": "||f_Delta - f_exact||_2"}, inplace=True)
pivot_df

In the above table, we can see  how fast the error decreases when $t$ and $N$ increase.

## Bibliography

$[1]$ <t style="font-variant: small-caps">Mohammadi, M.</t>, Analysis of discretization schemes for Fokker-Planck equations and related optimality systems [doctoral thesis], 2015

$[2]$ <t style="font-variant: small-caps">Khasi, M., and Fereshteh S.</t>, Numerical solutions of Fokker–Planck Equation based on Hermite functions, Communications in Combinatorics, Cryptography & Computer Science, 2022


$[3]$ <t style="font-variant: small-caps">Kloeden, P. E. and Platen, E.</t>, Numerical solution of stochastic differential equations, Applications of Mathematics, 1992

$[4]$ <t style="font-variant: small-caps">Milshtein, G. N.</t>, Approximate Integration of Stochastic Differential Equations, Theory of Probability & Its Applications. 19 (3): 557–000, 1974

$[5]$ <t style="font-variant: small-caps">Boyd, J. P.</t>, Chebyshev and Fourier Spectral Methods, 1974